# Cat vs Dog

In this project, we will go through the process of creating a cat vs dog classifier using UpStride Classification API.

First, let's download the dataset by visiting the Kaggle competition https://www.kaggle.com/c/dogs-vs-cats/data and clicking the "download all" button.

Unzip the archive. You will find another zip file inside, "train.zip". Unzip this one also.

Now we can prepare the dataset by moving the images in two differents folder for the two classes

In [9]:
# change this path to match you directory
train_directory = '/home/seb/Downloads/dogs-vs-cats/train'
cat_directory = '/home/seb/Downloads/dogs-vs-cats/dataset/cat'
dog_directory = '/home/seb/Downloads/dogs-vs-cats/dataset/dog'

In [10]:
import os
import shutil

os.makedirs(cat_directory, exist_ok=True)
os.makedirs(dog_directory, exist_ok=True)

for image in os.listdir(train_directory):
    if image.startswith('cat'):
        shutil.copy(os.path.join(train_directory, image), cat_directory)
    else:
        shutil.copy(os.path.join(train_directory, image), dog_directory)

Now we can convert this dataset to tfrecord format. For this we will use the script tfrecord_writer.py in scripts directory

In [2]:
!cd ~/workspace/upstride/classification_training \
    && export PYTHONPATH=$PYTHONPATH:$(pwd) \
    && python3 scripts/tfrecord_writer.py \
      --tfrecord_dir_path /home/seb/Downloads/dogs-vs-cats/tfrecord \
      --name cat_dog_dataset \
      --preprocessing CENTER_CROP_THEN_SCALE \
      --image_size 256 256 \
      --data.images_dir_path /home/seb/Downloads/dogs-vs-cats/dataset/ \
      --data.split_names train validation \
      --data.split_percentages 0.7 0.3

2020-10-02 16:13:44.399899: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-10-02 16:13:44.399934: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Building tfrecord for dataset named cat_god_dataset .....
Found 2 classes
Creating train split.....
Done
Creating validation split.....
Done
Dataset creation complete and stored in /home/seb/Downloads/dogs-vs-cats/tfrecord


if you go to the directory /home/seb/Downloads/dogs-vs-cats/tfrecord you will find 4 files :
 - dataset_info.yaml
 - train_0.tfrecord
 - train_1.tfrecord
 - validation_0.tfrecord
 
in dataset_info you will see information regarding this dataset.

In this case, you can see that the training dataset has 17500 images and the validation 7500

Now we can start a real training :

In [4]:
!cd ~/workspace/upstride/classification_training \
    && python3 train.py \
      --model_name MobileNetV2 \
      --num_epochs 20 \
      --checkpoint_dir /tmp/checkpoint \
      --log_dir /tmp/log \
      --framework tensorflow \
      --export.dir /tmp \
      --dataloader.name cat_dog_dataset \
      --dataloader.data_dir /home/seb/Downloads/dogs-vs-cats/tfrecord \
      --input_size 32 32 3 \
      --dataloader.train_list RandomHorizontalFlip Normalize \
      --dataloader.val_list Normalize \
      --dataloader.batch_size 64 \
      --num_classes 10 \
      --optimizer.lr 0.0001

2020-10-02 16:38:22.352137: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-10-02 16:38:22.352166: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
{'dataloader': {'train_list': ['RandomHorizontalFlip', 'Normalize'], 'val_list': ['Normalize'], 'data_dir': '/home/seb/Downloads/dogs-vs-cats/tfrecord', 'name': 'cat_dog_dataset', 'train_split_id': 'train', 'val_split_id': 'validation', 'batch_size': 64, 'Normalize': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225], 'scale_in_zero_to_one': True, 'only_subtract_mean': False}, 'ColorJitter': {'brightness': 0.05, 'contrast': [0.7, 1.3], 'saturation': [0.6, 1.6], 'hue': 0.08, 'clip': [0.0, 1.0]}, 'RandomRotate': {'angle': 10, 'interpolation': 'nearest'}, 'CentralCrop': {'size': [224, 224], 'crop_p

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connection.py", line 156, in _new_conn
    conn = connection.create_connection(
  File "/usr/lib/python3/dist-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/usr/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 665, in urlopen
    httplib_response = self._make_request(
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/usr/lib/python3/dist-packages/urllib3/connectionpool.py", line 996, in _validate_conn
    conn.connect